In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models # add models to the list
from torchvision.utils import make_grid
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision.utils import save_image

In [43]:
torch.manual_seed(100)

In [44]:
print(torch.__version__)

2.1.1+cpu


In [45]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)
TRAIN_DIR = "data/train"
VAL_DIR = "data/val"
LEARNING_RATE = 2e-4
BATCH_SIZE = 20 #Use a number that can divide the total of photos- (here 961)
NUM_WORKERS = 2
IMAGE_SIZE = 256
CHANNELS_IMG = 3
L1_LAMBDA = 100
LAMBDA_GP = 10
NUM_EPOCHS = 40  #we suggest to start with 5
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_DISC = "disc.pth.tar"
CHECKPOINT_GEN = "gen.pth.tar"

Using device: cpu


In [46]:
#Discriminator model for Pix2Pix

class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(CNNBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, 1, bias=False, padding_mode="reflect"),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels * 2,features[0],kernel_size=4,stride=2,padding=1,padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2),
            )
            in_channels = feature

        layers.append(
            nn.Conv2d(
                in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"
            ),
        )

        self.model = nn.Sequential(*layers)

    def forward(self, x, y):
        x = torch.cat([x, y], dim=1)
        x = self.initial(x)
        return self.model(x)

In [47]:
#Generator model for Pix2Pix
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super(Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )

        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
        self.down = down

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x


class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )
        self.down1 = Block(features, features * 2, down=True, act="leaky", use_dropout=False)
        self.down2 = Block(features * 2, features * 4, down=True, act="leaky", use_dropout=False)
        self.down3 = Block(features * 4, features * 8, down=True, act="leaky", use_dropout=False)
        self.down4 = Block(features * 8, features * 8, down=True, act="leaky", use_dropout=False)
        self.down5 = Block(features * 8, features * 8, down=True, act="leaky", use_dropout=False)
        self.down6 = Block(features * 8, features * 8, down=True, act="leaky", use_dropout=False)
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features * 8, features * 8, 4, 2, 1,padding_mode="reflect"), nn.ReLU()
        )

        self.up1 = Block(features * 8, features * 8, down=False, act="relu", use_dropout=True)
        self.up2 = Block(features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True)
        self.up3 = Block(features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True)
        self.up4 = Block(features * 8 * 2, features * 8, down=False, act="relu", use_dropout=False)
        self.up5 = Block(features * 8 * 2, features * 4, down=False, act="relu", use_dropout=False)
        self.up6 = Block(features * 4 * 2, features * 2, down=False, act="relu", use_dropout=False)
        self.up7 = Block(features * 2 * 2, features, down=False, act="relu", use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features * 2, in_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], 1))
        up3 = self.up3(torch.cat([up2, d6], 1))
        up4 = self.up4(torch.cat([up3, d5], 1))
        up5 = self.up5(torch.cat([up4, d4], 1))
        up6 = self.up6(torch.cat([up5, d3], 1))
        up7 = self.up7(torch.cat([up6, d2], 1))
        return self.final_up(torch.cat([up7, d1], 1))

In [48]:
#Train function for Pix2Pix

torch.backends.cudnn.benchmark = True


def train_fn(disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, g_scaler, d_scaler,):
    loop = tqdm(loader, leave=True)

    for idx, (x, y) in enumerate(loop):
        x = x.to(DEVICE)
        y = y.to(DEVICE)

        # Train Discriminator
        with torch.cuda.amp.autocast():
            y_fake = gen(x)
            D_real = disc(x, y)
            D_fake = disc(x, y_fake.detach())
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train generator
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1_loss(y_fake, y) * L1_LAMBDA
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 10 == 0:
            loop.set_postfix(
                D_real=torch.sigmoid(D_real).mean().item(),
                D_fake=torch.sigmoid(D_fake).mean().item(),
            )

In [49]:
# Utils for Pix2Pix

def save_some_examples(gen, val_loader, epoch, folder):
  ##ADDITION
    y_pred = []
  ##

    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization#

        ##ADDITION
        y_prediction = y_fake.cpu().numpy() #convert pytorch tensor to numpy
        y_pred.append(y_prediction)
        ##

        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        if epoch == 1:
            save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")
    gen.train()

    ##ADDITION
    ypred = np.array(y_pred)
    #print(ypred)
    np.save("ypred.npy", ypred)
    ##

def save_some_examples_modified(gen, val_loader, epoch, folder, iteration):
  ##ADDITION
    y_pred = []
  ##
    
    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization#

        ##ADDITION
        y_prediction = y_fake.cpu().numpy() #convert pytorch tensor to numpy
        y_pred.append(y_prediction)
        ##

        save_image(y_fake, folder + f"/y_gen_{epoch}_{i}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}_{i}.png")
        if epoch == 1:
            save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}_{i}.png")
    gen.train()

    ##ADDITION
    ypred = np.array(y_pred)
    #print(ypred)
    np.save(f"ypred_{i}.npy", ypred)
    ##

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=config.DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

### SUPERSAMPLING

**Create the training set**

In [92]:
#import data
df = pd.read_csv('50m 3.12.csv')
df.head()

,id,SpaceSyntax_Choice_50m_None,Supermarkets_50m_id_count,noise final50m-sum_noise sum,BuilingArea_50m_None,overlap
0,1,NaN,NaN,NaN,311.0,NaN
1,2,NaN,NaN,NaN,101.0,NaN
2,3,NaN,NaN,NaN,2378.0,NaN
3,4,NaN,NaN,NaN,2501.0,NaN
4,5,NaN,NaN,NaN,689.0,NaN


In [93]:
df.fillna(0, inplace=True)

In [94]:
df.isnull().values.any()

False

In [96]:
df.columns

Index(['id', 'SpaceSyntax_Choice_50m_None', 'Supermarkets_50m_id_count',
       'noise final50m-sum_noise sum', 'BuilingArea_50m_None', 'overlap'],
      dtype='object')

In [98]:
#Transform to numpy
data=np.array(df[['id','SpaceSyntax_Choice_50m_None', 'Supermarkets_50m_id_count',
       'noise final50m-sum_noise sum']])
data.shape

(150000, 4)

In [99]:
#Because of the id system in python , width and height are opposite to QGIS
w=400 #width
h=375 #height
s=256 #size
step=32 #step
cutoff=400 # how much overlap with the test area

In [100]:
data=np.transpose(data)

data= data.reshape(4,h,w)
data.shape


(4, 375, 400)

In [101]:
#Divide into set

idlist=[]

for j in range(s//2-1,w-s//2,step):
    for i in range(s//2-1,h-s//2,step):
        idlist.append(data[0,i,j])


dataclip=np.empty((0,3,256,256))

for id in idlist:
    #print(id)
    
    row=int(id//w) #UPDATE
    #print(row)

    col=int(id-w*row -1) #UPDATE
    #print(col)
    
    #print(s)
    minRow = row-s//2+1
    maxRow = row+s//2+1
    minCol = col-s//2+1
    maxCol = col+s//2+1

    #remove id column because we do not need it further
    sample= data[1:,minRow:maxRow,minCol:maxCol]

    #print(sample.shape)
    dataclip=np.append(dataclip,[sample],axis=0)


In [102]:
dataclip.shape

(20, 3, 256, 256)

In [103]:
#Normalize the data

#Normalize R
dataclip[:,0,:,:] = dataclip[:,0,:,:]/np.max(dataclip[:,0,:,:]) 

#Normalize G
dataclip[:,1,:,:] = dataclip[:,1,:,:]/np.max(dataclip[:,1,:,:]) 

#Normalize B
dataclip[:,2,:,:] = dataclip[:,2,:,:]/np.max(dataclip[:,2,:,:]) 

**Repeat the process for labels of training set**

In [104]:
#Transform to numpy
data=np.array(df[['id', 'BuilingArea_50m_None']])
data.shape

(150000, 2)

In [105]:
data=np.transpose(data)

data= data.reshape(2,h,w)
data.shape

(2, 375, 400)

In [106]:
ylabel=np.empty((0,1,256,256))

for id in idlist:
    #print(id)

    row=int(id//w)#UPDATE
    #print(row)

    col=int(id-w*row -1)#UPDATE
    #print(col)

    #print(s)
    minRow = row-s//2+1
    maxRow = row+s//2+1
    minCol = col-s//2+1
    maxCol = col+s//2+1

    #remove id column because we do not need it further
    sample= data[1:,minRow:maxRow,minCol:maxCol]

    #print(sample.shape)
    ylabel=np.append(ylabel,[sample],axis=0)

In [107]:
ylabel.shape

(20, 1, 256, 256)

In [108]:
#Normalize
ylabel[:,0,:,:] = ylabel[:,0,:,:]/np.max(ylabel[:,0,:,:]) 

### REMOVE THE OVERLAPING IMAGES WITH TEST SET

In [109]:
#Transform to numpy
data=np.array(df[['id','overlap']])
data.shape

(150000, 2)

In [110]:
data=np.transpose(data)

data= data.reshape(2,h,w)
data.shape

(2, 375, 400)

In [111]:
overlaplabel=np.empty((0,1,256,256))

for id in idlist:
    #print(id)

    row=int(id//w) #UPDATE
    #print(row)

    col=int(id-w*row -1) #UPDATE
    #print(col)

    #print(s)
    minRow = row-s//2+1
    maxRow = row+s//2+1
    minCol = col-s//2+1
    maxCol = col+s//2+1

    #remove id column because we do not need it further
    sample= data[1:,minRow:maxRow,minCol:maxCol]

    #print(sample.shape)
    
    overlaplabel=np.append(overlaplabel,[sample],axis=0)

In [112]:
#Find the sum of Overlap for each image 
overlaplabelSum=np.sum(np.sum(overlaplabel[:,0],axis=1),axis=1)

#Select the images that only have a small overlap 
dataclipNew=dataclip[np.where(overlaplabelSum<cutoff)]
ylabelNew=ylabel[np.where(overlaplabelSum<cutoff)]

In [113]:
#check shape of X data train
dataclipNew.shape

(5, 3, 256, 256)

In [114]:
#recreating 3 channels in total
rgb = np.hstack((ylabelNew,ylabelNew,ylabelNew))
rgb.shape

(5, 3, 256, 256)

### Create the test set / validation set

In [115]:
#import data
dft = pd.read_csv('10m 3.12.csv')
dft.head()

,id,SpaceSyntax_10m_choice_ASA216_T10,Supermarkets_10m (1)_id_count,noise sum_10m_NoiseClass10m,BuilingArea_10m_None
0,1,NaN,NaN,57.5,72.0
1,2,NaN,NaN,57.5,39.0
2,3,NaN,NaN,57.5,87.0
3,4,5946.0,NaN,57.5,26.0
4,5,5946.0,NaN,NaN,1.0


In [116]:
dft.columns

Index(['id', 'SpaceSyntax_10m_choice_ASA216_T10',
       'Supermarkets_10m (1)_id_count', 'noise sum_10m_NoiseClass10m',
       'BuilingArea_10m_None'],
      dtype='object')

In [117]:
dft.fillna(0, inplace=True)

In [118]:
#Transform to numpy
datatest=np.array(dft[['id', 'SpaceSyntax_10m_choice_ASA216_T10',
       'Supermarkets_10m (1)_id_count', 'noise sum_10m_NoiseClass10m']])
datatest.shape

datatest=np.transpose(datatest)

datatest= datatest.reshape(4,300,300)
datatest.shape

(4, 300, 300)

In [119]:
#Divide into set

w=300 #width
h=300 #height
s=256 #size
step=32 #step


idlisttest=[]

for i in range(s//2-1,w-s//2,step):
    for j in range(s//2-1,h-s//2,step):
        idlisttest.append(datatest[0,i,j])

print(idlisttest)



[38228.0, 38260.0, 47828.0, 47860.0]


In [120]:
datacliptest=np.empty((0,3,256,256))
##UPDATE
idvaltest=np.empty((0,256,256))

for id in idlisttest:
    #print(id)

    row=int(id//h)
    #print(row)

    col=int(id-h*row -1)
    #print(col)

    #print(s)
    minRow = row-s//2+1
    maxRow = row+s//2+1
    minCol = col-s//2+1
    maxCol = col+s//2+1

    #remove id column because we do not need it further
    sample= datatest[1:,minRow:maxRow,minCol:maxCol]
    idval=datatest[0,minRow:maxRow,minCol:maxCol]

    #print(sample.shape)
    datacliptest=np.append(datacliptest,[sample],axis=0)
    idvaltest=np.append(idvaltest,[idval],axis=0)

In [121]:
datacliptest.shape

(4, 3, 256, 256)

In [122]:
#Normalize the data

#Normalize R
datacliptest[:,0,:,:] = datacliptest[:,0,:,:]/np.max(datacliptest[:,0,:,:]) 

#Normalize G
datacliptest[:,1,:,:] = datacliptest[:,1,:,:]/np.max(datacliptest[:,1,:,:]) 

#Normalize B
datacliptest[:,2,:,:] = datacliptest[:,2,:,:]/np.max(datacliptest[:,2,:,:]) 

**TRAIN THE MODEL**

In [123]:
#Configurations

disc = Discriminator(in_channels=3).to(DEVICE)
gen = Generator(in_channels=3, features=64).to(DEVICE)
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999),)
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
BCE = nn.BCEWithLogitsLoss()
L1_LOSS = nn.L1Loss()

In [124]:
#Train the model
from torch.utils.data import TensorDataset, DataLoader

##ADDITION
tensor_x = torch.Tensor(dataclipNew) # transform to torch tensor
tensor_y = torch.Tensor(rgb)

train_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
##

train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=NUM_WORKERS,)

g_scaler = torch.cuda.amp.GradScaler()
d_scaler = torch.cuda.amp.GradScaler()


for epoch in range(NUM_EPOCHS):
    train_fn(disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler)
    
    if SAVE_MODEL and epoch % 5 == 0:
        save_checkpoint(gen, opt_gen, filename = CHECKPOINT_GEN)
        save_checkpoint(disc, opt_disc, filename = CHECKPOINT_DISC)

save_checkpoint(gen, opt_gen, filename = CHECKPOINT_GEN)
save_checkpoint(disc, opt_disc, filename = CHECKPOINT_DISC)
        
# Ideally we need D_fake=0 and D_real=1

100%|████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.97s/it, D_fake=0.533, D_real=0.428]


=> Saving checkpoint
=> Saving checkpoint


100%|████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.11s/it, D_fake=0.407, D_real=0.583]


=> Saving checkpoint
=> Saving checkpoint


100%|████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.79s/it, D_fake=0.396, D_real=0.671]


...

=> Saving checkpoint
=> Saving checkpoint


100%|████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.02s/it, D_fake=0.249, D_real=0.756]


=> Saving checkpoint
=> Saving checkpoint


100%|████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.96s/it, D_fake=0.195, D_real=0.794]


=> Saving checkpoint
=> Saving checkpoint


In [125]:
import os

# 创建名为 'final' 的文件夹在桌面上
desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
final_folder_path = os.path.join(desktop_path, 'final')
if not os.path.exists(final_folder_path):
    os.makedirs(final_folder_path)

# 循环遍历数据
for i in range(len(datacliptest)):
    tensor_x = torch.Tensor(datacliptest[i].reshape(1, 3, 256, 256))  # 转换为 torch 张量
    # 我们可以使用一个假的测试目标/我们只需要它作为占位符
    testRandomY = np.empty_like(datacliptest[i].reshape(1, 3, 256, 256))
    tensor_y = torch.Tensor(testRandomY)

    val_dataset = TensorDataset(tensor_x, tensor_y)  # 创建数据集

    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    # 保存文件到 'final' 文件夹中
    save_some_examples_modified(gen, val_loader, 5, folder=final_folder_path, iteration=i)  # 记得选择 epoch

**SAVE THE RESULTS**

In [126]:
nSamples = len(datacliptest)
yFull = np.zeros((nSamples,3,256,256))

for i in range(len(datacliptest)): 
    yFull[i] = np.load(f"ypred_{i}.npy")
    
yFull=yFull.reshape(1, 3, -1)
yFull.shape
idvaltest2=idvaltest.reshape(-1)

In [127]:
dfy=pd.DataFrame(idvaltest2[:],columns=['id'])
dfy['ch1']= yFull[0,0,:]
dfy['ch2']= yFull[0,1,:]
dfy['ch3']= yFull[0,2,:]
dfy.to_csv(f'./_pred_LG.csv',index=False)